In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import math
import seaborn as sns
from my_func import *

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from scipy import stats
from scipy.stats import kurtosis, skew

%matplotlib inline

# Step 1 Import Data

In [4]:
df = pd.read_csv('kc_house_data.csv')
df.shape #21,597 rows and 21 columns
df.head()



,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [5]:
print(df.columns)
print(df.info()) # Two object type columns that could be integers

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  float64
 9   view           21534 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          215

# Step 2 Explore the Data

### What does our raw data look like?
**id** - integer - this is a not entirely unique identifier ***drop this column***<br/>
**date** - object (switch to datetime) - We are assuming this is a sale date recorded in the county office <br/>
**price** - float - assuming this is the agreed upon sales price of the house <br/>
**bedrooms** - int - the number of bedrooms in the house <br/>
**bathrooms** - float - the number of bathrooms in the house <br/>
**sqft_living** - int - the living sf of the house often described as the finished area. Finished area is defined as "an enclosed area in a house at is suitable for year round use, embodying walls, floors, and ceiling that are similar to the rest of the house." <br/>
**sqft_lot** - int - the square footage of the parcel of land the house sits upon <br/>
**floors** - float - the number of floors in a house <br/>
**waterfront** - float (categorical) - binary value if house is on water or not with 1 being True and 0 being False <br/>
**view** - float - have no clue what this is? Maybe number of times a home was viewed by the appraiser? ***drop this column***<br/>
**condition** - int (categorical) - building condition scale relative to age and grade from 1-5 with 5 being the best (https://info.kingcounty.gov/assessor/esales/Glossary.aspx?type=r)<br/>
**grade** - int (categorical) - Represents the construction quality of improvements. Grades run from grade 1 to 13 with 13 being the best (https://info.kingcounty.gov/assessor/esales/Glossary.aspx?type=r)<br/>
**sqft_above** - int - living area sf above grade<br/>
**sqft_basement** - object (need to switch to int) - basement finished area sqft<br/>
**yr_built** - int - year the house was built<br/>
**yr_renovated** - float (should be int) - year house was renovated ***drop this column***<br/>
**zipcode** - int - zip code<br/>
**lat** - float64 - lattitude house is at? ***drop this column***<br/>
**long** - float - longitude of house? ***drop this column***<br/>
**sqft_living15** - sqft of living area for 15 nearest neighbors<br/>
**sqft_lot15** - int - sqft of land house is on for 15 nearest neighbors<br/> 



## 2.1 Drop Useless columns

We have decided to drop the ID, view, yr_renovated, lat, and long columns right off the bat for various reasons.
- The ID column has no particular value and is not a completely unique identifier as some values appear more than once. 
- The view column in uniterpretable
- The yr_renovated column has inaccurate data and too many missing values
- the lat and long have been dropped due to it providing similar information as zipcode, just more granular

In [6]:
df.drop(columns = ['id','view','lat','long','yr_renovated'],inplace=True)

## 2.2 Null Values

In [7]:
missing_data(df)
#2,376 NaN values 11% of the total data

,data_type,pct_missing,missing_count
date,object,0.000,0
price,float64,0.000,0
bedrooms,int64,0.000,0
bathrooms,float64,0.000,0
sqft_living,int64,0.000,0
sqft_lot,int64,0.000,0
floors,float64,0.000,0
waterfront,float64,11.002,2376
condition,int64,0.000,0
grade,int64,0.000,0


In [8]:
df.waterfront = df.waterfront.fillna(0) #fill NaN values with 0
df.waterfront.value_counts()

0.0    21451
1.0      146
Name: waterfront, dtype: int64

**Actions:** The waterfront column has 2,376 values that are NaN. We decided to impute these values with 0 because more than likely if the property was on the waterfront it would be noted.

In [7]:
df_value_counts(df) #this function prints out all the columns value counts to check the values for anomaliesb

**************** Column Name: date ****************
6/23/2014    142
6/25/2014    131
6/26/2014    131
7/8/2014     127
4/27/2015    126
            ... 
11/2/2014      1
8/3/2014       1
7/27/2014      1
5/15/2015      1
1/31/2015      1
Name: date, Length: 372, dtype: int64
----------------------------------------------------------------------------------------------------

**************** Column Name: price ****************
350000.0    172
450000.0    172
550000.0    159
500000.0    152
425000.0    150
           ... 
870515.0      1
336950.0      1
386100.0      1
176250.0      1
884744.0      1
Name: price, Length: 3622, dtype: int64
----------------------------------------------------------------------------------------------------

**************** Column Name: bedrooms ****************
3     9824
4     6882
2     2760
5     1601
6      272
1      196
7       38
8       13
9        6
10       3
11       1
33       1
Name: bedrooms, dtype: int64
---------------------------------

In [9]:
df['sqft_basement'] = df['sqft_basement'].replace('?','0.0')
df.sqft_basement.value_counts() #no more ? values because we replaced them with 0 (the median)

0.0       13280
600.0       217
500.0       209
700.0       208
800.0       201
          ...  
518.0         1
65.0          1
1248.0        1
1275.0        1
1880.0        1
Name: sqft_basement, Length: 303, dtype: int64

## 2.3 Data Types

In [10]:
df.date = pd.to_datetime(df.date) #convert date column to type datetime
df['sqft_basement'] = df['sqft_basement'].astype(float) #convert from object data type to float
df.set_index('date',inplace=True)
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,condition,grade,sqft_above,sqft_basement,yr_built,zipcode,sqft_living15,sqft_lot15
date,,,,,,,,,,,,,,,
2014-10-13,221900.0,3,1.00,1180,5650,1.0,0.0,3,7,1180,0.0,1955,98178,1340,5650
2014-12-09,538000.0,3,2.25,2570,7242,2.0,0.0,3,7,2170,400.0,1951,98125,1690,7639
2015-02-25,180000.0,2,1.00,770,10000,1.0,0.0,3,6,770,0.0,1933,98028,2720,8062
2014-12-09,604000.0,4,3.00,1960,5000,1.0,0.0,5,7,1050,910.0,1965,98136,1360,5000
2015-02-18,510000.0,3,2.00,1680,8080,1.0,0.0,3,8,1680,0.0,1987,98074,1800,7503


# Step 3 Export Cleaned Data

In [12]:
df.to_csv('kc_house_data_cleaned.csv',index=False)

0.0    21451
1.0      146
Name: waterfront, dtype: int64